In [1]:
import transformers
import torch
import os
import json

In [2]:
ROOT = ".."
PROMPT_PATH = "../prompts/"
MAX_NEW_TOKENS = 50
TEMPERATURE = 0.99
N_OUTPUTS = 100

In [3]:
with open(os.path.join(ROOT, 'token.txt'), 'r') as f:
    HF_TOKEN = f.read()

In [3]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token=HF_TOKEN
)

Device set to use cuda:0


In [ ]:
#  List of prompts
prompts = [
    "Explain the concept of deep learning.",
    "What are the main differences between Python and Java?",
    "Describe the process of photosynthesis."
]

# Generate responses for each prompt
for prompt in prompts:
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )
    
    print(f"Prompt: {prompt}")
    print(f"Response: {outputs[0]['generated_text']}")
    print("-" * 50)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Explain the concept of deep learning.
Response: [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Explain the concept of deep learning.'}, {'role': 'assistant', 'content': "Deep learning is a subfield of machine learning that involves the use of neural networks with multiple layers to analyze and interpret data. These neural networks are inspired by the structure and function of the human brain, where neurons communicate with each other through complex networks of connections.\n\nIn traditional machine learning, algorithms are designed to recognize patterns in data using simple rules or statistical models. However, deep learning algorithms can learn more complex patterns and relationships in data, especially when the data is high-dimensional and noisy.\n\nHere's a simplified overview of how deep learning works:\n\n1. **Data Preparation**: The first step is to prepare the data for use in the neural network. This includes normalizing the

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: What are the main differences between Python and Java?
Response: [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'What are the main differences between Python and Java?'}, {'role': 'assistant', 'content': "Python and Java are two popular programming languages that have distinct differences in their design, syntax, and use cases. Here are the main differences:\n\n**1. Syntax and Structure:**\n\t* Python: Known for its concise and readable syntax, Python uses indentation to define code blocks, making it easier to read and write. It has a more flexible and dynamic nature.\n\t* Java: Has a more traditional, structured syntax, with a focus on object-oriented programming (OOP). It uses keywords like `public`, `private`, and `static` to define classes and variables.\n\n**2. Type System:**\n\t* Python: Dynamically-typed, which means it doesn't require explicit type definitions for variables. It uses a type system based on runtime checks.\n\t*

In [15]:
dataset = "dtd"

with open(os.path.join(PROMPT_PATH, f"{dataset}/{dataset}_prompts_full.json"), 'r') as f:
    templates = json.load(f)
with open(os.path.join(PROMPT_PATH, f"meta_prompts.json"), 'r') as f:
    meta_prompts = json.load(f)[dataset]

classnames = list(templates[dataset].keys())
print(f"dataset {dataset} has {len(classnames)} classes and {len(meta_prompts)} meta-prompts.")

dataset dtd has 47 classes and 5 meta-prompts.


In [33]:
instructions = """Please format your response as one that contains only lower case letters and no special characters (including new lines, bold, and any markdown artifacts) other than a period ('.') or commas (','). 
The response should be a single sentence ending in a period that is directed toward the final instruction in this message. Your sentence should be a minimum of three words and maximum of thirty."""

num_seeds = N_OUTPUTS // len(meta_prompts)
for class_ in [classnames[0]]:
    prompts = [p.replace("{c}", class_) for p in meta_prompts]
    for prompt in prompts:
        messages = [
            {"role": "user", "content": instructions + " " + prompt}
        ]
        for seed in range(num_seeds):
            torch.manual_seed(seed)
            outputs = pipeline(
                messages,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=True,
                temperature=TEMPERATURE,
                top_p=0.9,
                pad_token_id=128001
            )
            print(outputs[0]['generated_text'][1]['content'])

gauzy material appears to be a thin and delicate fabric often made of silk or cotton and commonly used in clothing and upholstery.
gauzy material is often thin and translucent.
gauzy material resembles a soft blend of silk and cotton, often with a loose weave and a subtle sheen to it.
gauzy material is a thin and lightweight fabric often used in decorative arts and home textiles it has a soft and airy texture and may be colored or patterned with intricate designs.
gauzy material has a loose and airy texture that is often found in fabrics such as chiffon or organza.
gauzy material is often made from delicate fabrics like cotton or silk that have been treated with chemicals to give them a soft and airy texture and can be very fragile and prone to wrinkles.
gauzy material is often described as translucent or semi-transparent in nature usually appearing soft and delicate often associated with flowy fabrics or lightweight textiles.
gauzy material is usually thin and very light and delicate 

KeyboardInterrupt: 